# Nozzle Gas Transition Test

# Import

In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib as mpl
import os
import imageio.v3 as iio  # pip install imageio
import numpy as np
import math
from PIL import Image, ImageDraw, ImageFont
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap

# Geometry

In [ ]:
#from numba import jit, njit
one=time.time()
### generate mesh ###
# Controller
dx = 1; dt = 1
S = dx / dt
total_time = 2000#500#2000#1.0e4
cs = S / cp.sqrt(3)
# L = 200
# W = 100
# H = 100
L = 220
W = 50
H = 50
######################### 
x = cp.arange(0, L, dx); y = cp.arange(0, W, dx); z = cp.arange(0, H, dx)
nx = len(x); ny = len(y); nz = len(z)
dof = nx * ny * nz
YYY, ZZZ = cp.meshgrid(y, z); XXX, ZZZ = cp.meshgrid(x, z)
X, Y, Z = cp.meshgrid(x, y, z)
steps = int(total_time / dt)
print('nx, ny, nz:', nx, ny, nz); print('total steps:', steps)
wall_start = 40#nx // 2 - 20#20
wall_end   = nx#120# nx // 2 + 20

In [ ]:
def generate_particles_optimized(particles, nx, ny, nz, radius_factor, gap, wall_end):
    radius = nx * radius_factor
    spacing = gap * radius #1.8 * radius
    r_sq = radius ** 2 
    start_x = max(0 + spacing/3, wall_end + 70)
    
    #x_positions = cp.arange(start_x + spacing/2, nx - spacing/2, spacing)
    #x_positions = cp.arange(start_x + spacing/2, nx , spacing)
    #y_positions = cp.arange(0 + spacing/3, ny - spacing/3, spacing)
    #z_positions = cp.arange(0 + spacing/3, nz - spacing/3, spacing)
    x_positions = cp.arange(start_x , nx , spacing)
    y_positions = cp.arange(0 , ny , spacing)
    z_positions = cp.arange(0 , nz , spacing)
    Z, Y, X = cp.ogrid[:nz, :ny, :nx]

    centers_x = x_positions.get()
    centers_y = y_positions.get()
    centers_z = z_positions.get()

    for cx in centers_x:
        i_min = max(int(cx - radius), 0)
        i_max = min(int(cx + radius) + 1, nx)
        
        slice_x = X[:, :, i_min:i_max]
        dist_x_sq = (slice_x - cx) ** 2

        for cy in centers_y:
            j_min = max(int(cy - radius), 0)
            j_max = min(int(cy + radius) + 1, ny)
            
            slice_y = Y[:, j_min:j_max, :]
            dist_y_sq = (slice_y - cy) ** 2

            for cz in centers_z:
                k_min = max(int(cz - radius), 0)
                k_max = min(int(cz + radius) + 1, nz)
                dist_z_sq = (Z[k_min:k_max, :, :] - cz) ** 2
                dist_sq = dist_z_sq + dist_y_sq + dist_x_sq
                particles[k_min:k_max, j_min:j_max, i_min:i_max] |= (dist_sq <= r_sq)

    return particles

# GIF

In [ ]:
def VTKConverter(step, nx, ny, nz, u, T, rho, newP, index):
    lines = []
    lines.append('# vtk DataFile Version 2.0\n')
    lines.append('stream with structured data\n')
    lines.append('\n')
    lines.append('ASCII\n')
    lines.append('DATASET STRUCTURED_POINTS\n')
    lines.append('DIMENSIONS ' + str(nx) + ' ' + str(ny) + ' ' + str(nz) + ' ' + '\n')
    lines.append('ORIGIN ' + str(0) + ' ' + str(0) + ' ' + str(0) + '\n')
    lines.append('SPACING ' + str(1) + ' ' + str(1) + ' ' + str(1) + '\n')
    lines.append('\n')
    lines.append('POINT_DATA ' + str(nx * ny * nz) + '\n')
    
    lines.append('SCALARS T float\n')
    lines.append('LOOKUP_TABLE default\n')
    T_flat = cp.asnumpy(T.ravel())
    lines.append('\n'.join(map(str, T_flat)) + '\n')

    lines.append('SCALARS rho float 1\n')
    lines.append('LOOKUP_TABLE default\n')
    rho_flat = cp.asnumpy(rho.ravel())
    lines.append('\n'.join(map(str, rho_flat)) + '\n')

    lines.append('SCALARS newP float 1\n')
    lines.append('LOOKUP_TABLE default\n')
    pressure_flat = cp.asnumpy(newP.ravel())
    lines.append('\n'.join(map(str, pressure_flat)) + '\n')

    lines.append('SCALARS Phase float 1\n')
    lines.append('LOOKUP_TABLE default\n')
    index_flat = cp.asnumpy(index.ravel())
    lines.append('\n'.join(map(str, index_flat)) + '\n')

    
    lines.append('VECTORS u float \n')
    u_flat = cp.asnumpy(u.reshape(3, -1))
    uu = ['{} {} {}'.format(u_flat[0, i], u_flat[1, i], u_flat[2, i]) for i in range(u_flat.shape[1])]
    lines.append('\n'.join(uu) + '\n')

    with open(f"./result_/f_Nozzle_lowPressureTc_at{int(step)}.vtk", 'w') as outFile:
        outFile.writelines(lines)
    return

def VTKConverter_Frame(step, nx, ny, nz, nozzles, particles):
    lines = []
    lines.append('# vtk DataFile Version 2.0\n')
    lines.append('stream with structured data\n')
    lines.append('\n')
    lines.append('ASCII\n')
    lines.append('DATASET STRUCTURED_POINTS\n')
    lines.append('DIMENSIONS ' + str(nx) + ' ' + str(ny) + ' ' + str(nz) + ' ' + '\n')
    lines.append('ORIGIN ' + str(0) + ' ' + str(0) + ' ' + str(0) + '\n')
    lines.append('SPACING ' + str(1) + ' ' + str(1) + ' ' + str(1) + '\n')
    lines.append('\n')
    lines.append('POINT_DATA ' + str(nx * ny * nz) + '\n')

    lines.append('SCALARS nozzles float 1\n')
    lines.append('LOOKUP_TABLE default\n')
    nozzles_flat = cp.asnumpy(nozzles.ravel())
    lines.append('\n'.join(map(str, nozzles_flat)) + '\n')

    lines.append('SCALARS particles float 1\n')
    lines.append('LOOKUP_TABLE default\n')
    particles_flat = cp.asnumpy(particles.ravel())
    lines.append('\n'.join(map(str, particles_flat)) + '\n')

    with open(f"./NozzleFrame_150Tc_at{int(step)}.vtk", 'w') as outFile:
        outFile.writelines(lines)
    return

In [ ]:

make_gif = True
gif_every = 10#200         
gif_slice = ("y", ny//2) 
gif_field = "index"       
vmin = 1; vmax = 4#460 
#vmin = None; vmax = None

# ======= quiver overlaid =======# 
overlay_quiver = True
quiver_every = 1          
quiver_stride = 4       
quiver_scale = None      
quiver_color = "k"      
quiver_alpha = 0.7
scale = 1
# ======= save =======# 
out_dir = "gif_frames"
gif_path = "Nozzle_rho.gif"
os.makedirs(out_dir, exist_ok=True)

frames = []              

def _get_slice(arr3d, axis_name, idx):
    if axis_name == "y":
        return arr3d[:, idx, :]
    if axis_name == "x":
        return arr3d[:, :, idx]
    if axis_name == "z":
        return arr3d[idx, :, :]
    raise ValueError("axis_name must be one of: 'x','y','z'")
    
def _to_rgb8(img2d, vmin=None, vmax=None, cmap_name=None):
    cmap = mpl.colormaps[cmap_name]
    img = img2d.astype(np.float64)
    if vmin is None: vmin = np.nanmin(img)
    if vmax is None: vmax = np.nanmax(img)
    norm = np.clip((img - vmin) / (vmax - vmin + 1e-12), 0.0, 1.0)
    cmap = mpl.colormaps[cmap_name]
    rgba = cmap(norm)
    return (rgba[:, :, :3] * 255).astype(np.uint8)

def draw_step(frame_rgb, step, color=(255,255,255)):
    img = Image.fromarray(frame_rgb)
    draw = ImageDraw.Draw(img)

    text = f"step = {step}"
    pos = (10, 10)  

    try:
        font = ImageFont.truetype("arial.ttf", 18)
    except:
        font = ImageFont.load_default()

    bbox = draw.textbbox(pos, text, font=font)
    draw.rectangle(bbox, fill=(1,1,1))
    draw.text(pos, text, fill=color, font=font)

    return np.array(img)

def add_step_bar(frame_rgb, step, C_time, bar_h=None, fg=(0,0,0), bg=(255,255,255)):
    """
    frame_rgb: (H, W, 3) uint8
    return: (H+bar_h, W, 3) uint8  (위쪽에 bar 추가)
    """
    img = Image.fromarray(frame_rgb)
    W, H = img.size
    cb_w=50  
    canvas = Image.new("RGB", (W, H + bar_h), color=bg)
    canvas.paste(img, (0, bar_h)) 

    draw = ImageDraw.Draw(canvas)
    text = f"step = {step * C_time / 1e-9} ns"
    try:
        font = ImageFont.truetype("arial.ttf", 400)
    except:
        font = ImageFont.load_default()
    bbox = draw.textbbox((0, 0), text, font=font)     
    tw = bbox[2] - bbox[0]
    th = bbox[3] - bbox[1]
    x = 10
    y = (bar_h - th) // 2
    draw.text((x, y), text, fill=fg, font=font)
    return np.array(canvas)

def add_colorbar_right(frame_rgb, vmin, vmax, cmap_name=None, cb_w=45, pad=0,
                       bg=(255,255,255), txt_color=(0,0,0)):
    """
    frame_rgb: (H, W, 3) uint8
    return: (H, W+cb_w+pad, 3) uint8
    """
    H, W, _ = frame_rgb.shape

    grad = np.linspace(1.0, 0.0, H)[:, None]
    grad = np.repeat(grad, cb_w, axis=1)

    cmap = mpl.colormaps[cmap_name]
    cb_rgb = (cmap(grad)[:, :, :3] * 255).astype(np.uint8)

    if pad > 0:
        pad_rgb = np.full((H, pad, 3), bg, dtype=np.uint8)
        out = np.concatenate([frame_rgb, pad_rgb, cb_rgb], axis=1)
        cb_x0 = W + pad
    else:
        out = np.concatenate([frame_rgb, cb_rgb], axis=1)
        cb_x0 = W

    img = Image.fromarray(out)
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()

    draw.text((cb_x0 + 4, 2), f"{vmax:.3g}", fill=txt_color, font=font)
    draw.text((cb_x0 + 4, H - 18), f"{vmin:.3g}", fill=txt_color, font=font)

    return np.array(img)

def fig_to_rgb(fig):
    fig.canvas.draw()
    buf = np.asarray(fig.canvas.buffer_rgba())
    return buf[:, :, :3]  # RGBA -> RGB

# def make_overlay_frame(field2d, X2d, Z2d, U2d, W2d, 
#                        vmin=None, vmax=None, cmap_name="viridis",
#                        stride=4, qscale=None, qcolor="w", qalpha=0.7,
#                        q_color_field=None, q_cmap="coolwarm", q_clim=None):
    
#     fig, ax = plt.subplots(figsize=(8, 8), dpi=120)

#     # 1. 배경 (Phase Index)
#     im = ax.imshow(field2d, origin="lower",
#                    vmin=vmin, vmax=vmax, cmap=cmap_name,
#                    extent=[X2d.min(), X2d.max(), Z2d.min(), Z2d.max()],
#                    aspect="equal")

#     # 2. 화살표 데이터 샘플링
#     xs = X2d[::stride, ::stride]
#     zs = Z2d[::stride, ::stride]
#     us = U2d[::stride, ::stride]
#     ws = W2d[::stride, ::stride]

#     # 3. 화살표 그리기 (색상 여부에 따라 분기)
#     if q_color_field is not None:
#         qs = q_color_field[::stride, ::stride]
#         q = ax.quiver(xs, zs, us, ws, qs, cmap=q_cmap, alpha=qalpha, scale=qscale)
#         q.set_clim(q_clim) # dT의 범위를 고정 (-limit, limit)
#     else:
#         ax.quiver(xs, zs, us, ws, color=qcolor, alpha=qalpha, scale=qscale)

#     ax.set_xticks([]); ax.set_yticks([])
#     fig.tight_layout(pad=0.2)

#     rgb = fig_to_rgb(fig)
#     plt.close(fig)
#     return rgb
def make_overlay_frame(field2d, X2d, Z2d, U2d, W2d,
                       vmin=None, vmax=None, cmap_name="viridis",
                       stride=4, qscale=None, qcolor="w", qalpha=1.0,
                       q_color_field=None, q_cmap="coolwarm", q_clim=None):
    fig, ax = plt.subplots(figsize=(10, 3), dpi=300)
    im = ax.imshow(field2d, origin="lower",
                   vmin=vmin, vmax=vmax, cmap=cmap_name,
                   extent=[X2d.min(), X2d.max(), Z2d.min(), Z2d.max()],
                   aspect="equal", alpha = 0.7)
    cbar1 = fig.colorbar(im, ax=ax, orientation = 'horizontal')
    cbar1.set_label('Phase (1:Gas, 2:Liq, 3:SC, 4: ice)') 
    cbar1.set_ticks([1, 2, 3, 4])
    cbar1.set_ticklabels(['Gas', 'Liquid', 'SC', 'ice'])
    cbar1.set_label('$\text{CO}_2$ Phase State', fontsize=12)
    xs = X2d[::stride, ::stride]; zs = Z2d[::stride, ::stride]
    us = U2d[::stride, ::stride]; ws = W2d[::stride, ::stride]
    if q_color_field is not None:
        qs = q_color_field[::stride, ::stride]
        q = ax.quiver(xs, zs, us, ws, qs, cmap=q_cmap, alpha=qalpha, scale=qscale)
        q.set_clim(q_clim)
        cbar2 = fig.colorbar(q, ax=ax, orientation = 'horizontal') # pad를 더 크게 줘서 겹치지 않게 함
        cbar2.set_label('dT (Temperature Change)')
    else:
        ax.quiver(xs, zs, us, ws, color=qcolor, alpha=qalpha, scale=qscale)
    ax.set_xticks([]); ax.set_yticks([])
    fig.tight_layout(pad=0.2)
    rgb = fig_to_rgb(fig)
    plt.close(fig)
    return rgb

# Property

고압에서 노즐을 통과하다가 저압영역이 되면서 단열팽창으로 인한 온도 하강 + 그냥 압력이낮아서? 


In [ ]:
print("========================================================================================================= Nondimensionalization")
Tc = 0.01 
Pc = 0.01 
Zc = 0.307
R = 1.0
rhoc = Pc / (Zc * R *Tc) 
a = 0.45727 * ((R**2) * (Tc**2) / Pc)
b = 0.07780 * R * Tc / Pc
print(f"{'a':<22}: {a:>8.3f} | "    f"{'b':<22}: {b:>8.3f} | "    f"{'R':<22}: {R:>8.3f} | "    f"{'Zc':<22}: {Zc:>6.3f}")
print(f"{'Tc':<22}: {Tc:>8.3f} | "    f"{'Pc':<22}: {Pc:>8.3f} | "    f"{'rhoc':<22}: {rhoc:>8.3f}")
print("========================================================================================================= Basic Momentum ")
vis_f = 0.05 
Pr = 0.71
vis_g = vis_f/Pr
tau_g = 3. * vis_g / (S ** 2) / dt + 0.5 
tau_f = 3. * vis_f / (S ** 2) / dt + 0.5 
print(f"{'vis_f':<22}: {vis_f:>8.3g} | "    f"{'vis_g':<22}: {vis_g:>8.3g}")
print(f"{'Pr':<22}: {Pr:>8.3g}")
print(f"{'tauF/dt':<22}: {tau_f/dt:>8.3g} | "    f"{'tau_g/dt':<22}: {tau_g/dt:>8.3g}")
print("========================================================================================================= Heat source ")
C_length = 1e-7 
C_u = 520
C_time = C_length / C_u 
C_rho = 468 / rhoc 
C_T = 304.13 / Tc 
C_P = 7.38e6 / Pc 
vis_real = 1e-7
print(f"{'C_rho [kg/m^3]':<22}: {C_rho:>8.3g} | "    f"{'C_T [k]':<22}: {C_T:>8.3g} | "    f"{'C_P [kg/m/s^2]':<22}: {C_P:>8.3g}")
print(f"{'C_length [m]':<22}: {C_length:>8.3g} | "    f"{'vis_real [m^2/s]':<22}: {vis_f * C_length **2 / C_time:>8.3g} | "    f"{'C_time [s]':<22}: {C_time:>8.3g}")
print(f"{'C_u [m/s]':<22}: {C_u:>8.3g}")
print("========================================================================================================= PR EOS ")
use_latent = True # PREOS에 기체-액체간 잠열은 잠재적으로 반영되어있음 그러나 dry ice변화는 반영되어있지않아서 추가로 latent heat를 더해줘야할 가능성이있음
latent_real = 90e3#180e3 # temperature, pressure decrease-> latent decrease
L_lb = latent_real / (C_u)**2 #0.01 (m/s)**2
use_work = True #
use_dampwork = False 
omega = 0.22394 #0.225 # 0.344 for water
Cv = 720 * (C_T / (C_u **2))
print(f"{'Cv in lu: ':<22}: {Cv:>8.3g}")
print(f"{'Real Cv [J/kg]':<22}: {Cv * (C_u**2/C_T):>8.3g} | "
      f"{'Ref latent [J/kg]':<22}: {latent_real:>8.3g}")
print(f"{'omega':<22}: {omega:>8.3f} | "    f"{'latent heat const':<22}: {L_lb:>8.3f} | "    f"{'use_latent':<22}: {use_latent} | "    f"{'use_work':<22}: {use_work} | "    f"{'use_dampwork':<15}: {use_dampwork}")
print("========================================================================================================= Initial bubble Density, Size")
high_rho = rhoc * 1.1
low_rho =  rhoc * 0.3 
print(f"{'rho ratio of liquid':<22}: {high_rho/rhoc:>8.3f} | "    f"{'rho ratio of gas':<22}: {low_rho/rhoc:>8.3f}")
print(f"{'rho_liquid_l':<22}: {high_rho:>8.3f} | "    f"{'rho_vapor_l':<22}: {low_rho:>8.3f}")
print(f"{'rho_liquid_l_real':<22}: {high_rho * C_rho:>8.3f} kg/m^3| "    f"{'rho_vapor_l_real':<22}: {low_rho * C_rho:>8.3f} kg/m^3")
print("========================================================================================================= SurfaceTension(ShenChen)")
forcing_method = "EDM" # "EDM" # "Guo"
G = -1 #-1.0 # 0.065/3 # D3Q19=6?
gwall = 0#-0.01 #0.445
beta = 1.16 #0.3 #1.16 # 0.886 for SC 
print(f"{'Forcing scheme':<22}: {forcing_method}")
print(f"{'G':<22}: {G:>8.3f} | "    f"{'gwall':<22}: {gwall:>8.3f}")
print(f"{'beta for SC correction':<22}: {beta:>8.3f}")
print("========================================================================================================= Initial Temperature")
T_inlet = 1.1 * Tc 
T_outlet = 1.1 * Tc
print(f"{'T ratio: ':<22}: {T_inlet/Tc:>8.3f}")
print(f"{'T_inlet':<22}: {T_inlet:>8.3f} | "    f"{'T_outlet':<22}: {T_outlet:>8.3f}")
print(f"{'T_inlet_real':<22}: {T_inlet * C_T :>8.3e} K | "    f"{'T_outlet_real':<22}: {T_outlet * C_T- 273.13:>8.3f} celcius")
print("========================================================================================================= In/outlet velocity")
inlet_vel = 0.0002
outlet_vel = 0.0002
print(f"{'inlet_vel':<22}: {inlet_vel:>8.3e} | "    f"{'outlet_vel':<22}: {outlet_vel:>8.3e}")
print(f"{'inlet_vel_real':<22}: {inlet_vel * C_u:>8.3f} m/s | "    f"{'outlet_vel':<22}: {outlet_vel * C_u:>8.3f} m/s")
use_particles = True; fixed_outlet = False # opened: False

# (basic) Array definitions

In [ ]:
######
Q = 19 
T_result = cp.ones((nz, ny, nx), dtype='f8') * T_inlet  # inital temperature
T_init = cp.ones((nz, ny, nx), dtype='f8') * T_inlet  # initial temperature
T_0 = cp.ones((nz, ny, nx), dtype='f8') * T_inlet  # initial temperature
rho0 = cp.ones((nz, ny, nx), dtype='f8') * low_rho  # initial density
rho_result = cp.ones((nz, ny, nx), dtype='f8') * low_rho # initial density
rho0[:, :, :wall_start] = high_rho
rho_result =  cp.copy(rho0)
rho_initial = cp.copy(rho0)
P = cp.zeros((3, nz, ny, nx), dtype='f8') * 0  # macroscopic momentum density
u = cp.zeros((3, nz, ny, nx), dtype='f8') * 0  # velocity
cohesion = cp.zeros((3, nz, ny, nx), dtype='f8') * 0  # molecule cohesion
Fwet = cp.zeros((3, nz, ny, nx), dtype='f8') * 0 
############################### 
# Vectorization
g = cp.zeros((Q, nz, ny, nx), dtype='f8')  # internal energy * rho
g_init = cp.zeros((Q, nz, ny, nx), dtype='f8')
g_c = cp.zeros((Q, nz, ny, nx), dtype='f8')
g_eq = cp.zeros((Q, nz, ny, nx), dtype='f8')
g_stream = cp.zeros((Q, nz, ny, nx), dtype='f8')
source_term = cp.zeros((Q, nz, ny, nx), dtype='f8')
f = cp.zeros((Q, nz, ny, nx), dtype='f8')
f_init = cp.zeros((Q, nz, ny, nx), dtype='f8')
f_c = cp.zeros((Q, nz, ny, nx), dtype='f8')
f_stream = cp.zeros((Q, nz, ny, nx), dtype='f8')
f_eq = cp.zeros((Q, nz, ny, nx), dtype='f8')
f_b = cp.zeros((Q, nz, ny, nx), dtype='f8')
f_cohesion = cp.zeros((Q, nz, ny, nx), dtype='f8')
#    x, y, z
c = cp.array([ #  (Wang et al., 2020)
    [0, 0, 0],  # 0
    [1, 0, 0],  # 1
    [-1, 0, 0], # 2
    [0, 1, 0],  # 3
    [0, -1, 0], # 4
    [0, 0, 1],  # 5
    [0, 0, -1], # 6
    [1, 1, 0], # 7
    [-1, -1, 0],  # 8
    [1, 0, 1], # 9
    [-1, 0, -1],# 10
    [0, 1, 1],  # 11
    [0, -1, -1], # 12
    [1, -1, 0], # 13
    [-1, 1, 0],# 14
    [1, 0, -1],  # 15
    [-1, 0, 1], # 16
    [0, 1, -1], # 17
    [0, -1, 1] # 18
], dtype='i8')

ai = cp.array([0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 12, 11, 14, 13, 16, 15, 18, 17], dtype='i8')
w = cp.array([1 / 3, 1 / 18, 1 / 18, 1 / 18, 1 / 18, 1 / 18, 1 / 18, 1 / 36, 1 / 36, 1 / 36, 1 / 36, 1 / 36, 1 / 36, 1 / 36, 1 / 36, 1 / 36, 1 / 36, 1 / 36, 1 / 36], dtype='f8')
cs2 = cs ** 2; cs4 = cs ** 4

zzz, yyy, xxx = cp.meshgrid(cp.arange(nz), cp.arange(ny), cp.arange(nx), indexing='ij')
outlet = cp.zeros((nz, ny, nx), dtype=cp.int32)
cy, cz = ny // 2, nz // 2
dist_sq = (yyy - cy)**2 + (zzz - cz)**2  # (nz,ny,nx)

# ---- nozzle x-range ----
x0 = int(wall_start)            # nozzle begins
x3 = int(wall_end)              # nozzle ends
x0 = max(x0, 1)
x3 = min(x3, nx - 2)
L  = max(x3 - x0, 1)

# ---- radii in lattice units (tune)  ----
r_in  = 10   # inlet radius
r_th  = 4    # throat radius
r_out = 15   # outlet radius (diffuser end)

# ---- throat location (tune) ----
x_th = x0 + int(0.35 * L)       # throat at 35% of nozzle length
x_th = min(max(x_th, x0 + 1), x3 - 1)

def smoothstep(t):
    t = cp.clip(t, 0.0, 1.0)
    return t*t*(3.0 - 2.0*t)

t1 = (xxx - x0) / (x_th - x0 + 1e-30)
r_conv = r_in + (r_th - r_in) * smoothstep(t1)
t2 = (xxx - x_th) / (x3 - x_th + 1e-30)
r_div  = r_th + (r_out - r_th) * smoothstep(t2)
r_x = cp.where(xxx <= x_th, r_conv, r_div)
in_nozzle_x = (xxx >= x0) & (xxx <= x3)
nozzle_wall = in_nozzle_x & (dist_sq > (r_x**2))
nozzles = nozzle_wall.astype(cp.float64)

plt.title('nozzle (CD wall mask)')
plt.imshow(cp.asnumpy(nozzles[:, ny//2, :][::-1]), cmap='jet')
plt.colorbar(); plt.show()

radius_inlet = r_in  # or r_in-1, r_in-2
hole_inlet = dist_sq <= (radius_inlet**2)   # (nz,ny,nx) but we'll use x=0 plane
################################################
particles = cp.zeros((nz, ny, nx), dtype='bool')
particles = cp.asarray(particles)
rad = 0.018; gap = 2.3
if use_particles == True: 
    particles = generate_particles_optimized(particles, nx, ny, nz, rad, gap, wall_end)
    particles = np.save("porous.npy",particles)
    particles = np.load("porous.npy")
    particles = cp.array(particles)
particles = particles.astype(float)
particles_mask = particles
plt.title('particles')
plt.imshow(cp.asnumpy(particles[:,ny//2,:][::-1]), cmap='jet')    
plt.colorbar(); plt.show()

VTKConverter_Frame(1, nx, ny, nz, nozzles, particles)
total = nz * ny * nx
diam = rad * 2

isn = cp.zeros((nz, ny, nx), dtype='i8')
isn_inlet = cp.zeros((nz, ny, nx), dtype='i8')
isn_inlet[:, :, 0] = 6  # left
isn[:, 0, :] = 3  # front
isn[:, -1, :] = 4  # back
isn[0, :, :] = 1  # bottom
isn[-1, :, :] = 2  # top
isn[:, :, -1] = 5  # right
isn[:, :, 0] = 6  # left
isn[:, 0, 0] = 7; isn[:, -1, 0] = 7; isn[:, 0, -1] = 7; isn[:, -1, -1] = 7
isn[0, :, 0] = 7; isn[-1, :, 0] = 7; isn[0, :, -1] = 7; isn[-1, :, -1] = 7
isn[0, 0, :] = 7; isn[-1, 0, :] = 7; isn[0, -1, :] = 7; isn[-1, -1, :] = 7

isn_binary = cp.where(isn == 0, 0, 1)
plt.imshow(cp.asnumpy(isn[:,ny//2,:][::-1])); plt.colorbar();plt.show()

bottom = cp.where(isn == 1); top = cp.where(isn == 2)
left = cp.where(isn == 6); right = cp.where(isn == 5)
front = cp.where(isn == 3); back = cp.where(isn == 4)
corner = cp.where(isn == 7); nozzles = cp.where(nozzles==1); outlet_ = cp.where(outlet == 1);  particles = cp.where(particles==1);
inlet_mask =  (isn_inlet == 6)  #& hole_inlet & (particles_mask == 0)#(isn == 6) 
inlet_idx  = cp.where(inlet_mask)

plt.title('inlet_mask')
plt.imshow(cp.asnumpy(inlet_mask[:,ny//2,:][::-1]), cmap='jet')    
plt.colorbar(); plt.show()

bottom_ = cp.array(bottom); top_ = cp.array(top)
left_ = cp.array(left); right_ = cp.array(right)
back_ = cp.array(back); front_ = cp.array(front)

top_insul = top_.copy(); top_insul[0] = top_[0]-1
bottom_insul = bottom_.copy(); bottom_insul[0] = bottom_[0]+1
back_insul = back_.copy(); back_insul[1] = back_[1]-1
front_insul = front_.copy(); front_insul[1] = front_[1]+1
left_insul = left_.copy(); left_insul[2] = left_[2]+1
right_insul = right_.copy(); right_insul[2] = right_[2]-1

bottom_isn_indices = cp.where(isn == 1); top_isn_indices = cp.where(isn == 2)
left_isn_indices = cp.where(isn == 6); right_isn_indices = cp.where(isn == 5)
back_isn_indices = cp.where(isn == 4); front_isn_indices = cp.where(isn == 3)
corner_isn_indices = cp.where(isn == 7); outlet_isn_indices = cp.where(outlet == 1)

BTiz_indices, BTix_indices, BTiy_indices = bottom_isn_indices
TPiz_indices, TPix_indices, TPiy_indices = top_isn_indices
LFiz_indices, LFix_indices, LFiy_indices = left_isn_indices
RTiz_indices, RTix_indices, RTiy_indices = right_isn_indices
BKiz_indices, BKix_indices, BKiy_indices = back_isn_indices
FRiz_indices, FRix_indices, FRiy_indices= front_isn_indices
CNiz_indices, CNix_indices, CNiy_indices= corner_isn_indices

params = dict(
    dx=dx, dt=dt, S=S,
    cs=cs, cs2=cs2, cs4=cs4,
    Q=Q,
    a=a, b=b, R=R, Tc=Tc,
    omega=omega,
    G=G, gwall=gwall,
    tau_f=tau_f, tau_g=tau_g,
    L_lb=L_lb, beta = beta,
    backT=T_inlet,
    high_rho = high_rho, low_rho = low_rho,
    T_inlet = T_inlet, T_outlet = T_outlet,
    use_latent = use_latent, use_work = use_work,
    use_dampWork = use_dampwork, fixed_outlet = fixed_outlet,
    divu_method = "u", eps = 1e-10,
    Cv = Cv,
    forcing_method = forcing_method)

geom = dict(
    nx=nx, ny=ny, nz=nz,
    c=c, ai=ai, w=w,
    XXX=XXX, ZZZ=ZZZ, YYY=YYY,  # plot용
    isn=isn, isn_binary=isn_binary,
    nozzles=nozzles, outlet_=outlet_, particles = particles,
    inlet_mask = inlet_mask,
    bottom_isn_indices=bottom_isn_indices,
    top_isn_indices=top_isn_indices,
    left_isn_indices=left_isn_indices,
    right_isn_indices=right_isn_indices,
    front_isn_indices=front_isn_indices,
    back_isn_indices=back_isn_indices,
    corner_isn_indices=corner_isn_indices)

state = dict(
    f=f, g=g,
    f_b=f_b, f_stream=f_stream, g_stream=g_stream,
    rho=rho_result, T=T_result,
    P=P, u=u,
    cohesion=cohesion, Fwet=Fwet)


# Initialization

In [ ]:
rho0[:,:,0] = high_rho
############# Initialize ##############################
for i in range(Q):
    cu = c[i, 0] * S * u[0] + c[i, 1] * u[1] * S + c[i, 2] * u[2] * S
    f[i] = rho0 * w[i] * (1 + (cu / cs ** 2) + (cu ** 2 / 2 * cs ** 4) - (u[0] * u[0] + u[1] * u[1] + u[2] * u[2]) / (2 * cs ** 2))
    #g[i] = w[i] * T_0 * (1 + (cu / cs ** 2))
    g[i] = w[i] * T_0 * (1 + (cu / cs**2) + (cu**2 / (2*cs**4)) - (u[0] * u[0] + u[1] * u[1] + u[2] * u[2]) / (2*cs**2))
rho_result = cp.sum(f, axis=0)
T_result = cp.sum(g, axis=0)
P[0] = 0; P[1] = 0; P[2] = 0
for i in range(Q):
    P[0] += f[i] * c[i, 0] * S # x momentum density
    P[1] += f[i] * c[i, 1] * S # y momentum density
    P[2] += f[i] * c[i, 2] * S # y momentum density
u = P / rho_result
###########
plt.title('initial T')
plt.imshow(cp.asnumpy(T_result[:,ny//2,:][::-1]), cmap='jet')    
plt.colorbar(); plt.show()
plt.title('initial P')
plt.imshow(cp.asnumpy(rho_result[:,ny//2,:][::-1] * cs **2), cmap='jet')    
plt.colorbar(); plt.show()
plt.title('initial rho')
plt.imshow(cp.asnumpy(rho_result[:,ny//2,:][::-1]), cmap='jet')    
plt.colorbar(); plt.show()
print(f'initial density field: {cp.min(rho_result * C_rho)} kg/m^2, {cp.max(rho_result * C_rho)} MPa,')


# Surface tension (solid cohesion)

In [ ]:
def solid_to_cohesion(rho, geom, params, cohesion, Fwet):
    """Fwet = -gwall * rho * sum_j w_j * isn_binary * c_j"""
    c = geom["c"]; w = geom["w"]; isn_binary = geom["isn_binary"]
    cohesion[:] = 0
    Q = params["Q"]
    for j in range(Q):
        cohesion[0] += w[j] * isn_binary * c[j, 0]
        cohesion[1] += w[j] * isn_binary * c[j, 1]
        cohesion[2] += w[j] * isn_binary * c[j, 2]
    Fwet[:] = -params["gwall"] * rho * cohesion
    return Fwet

# PR EOS

In [ ]:
def alphaT(T, params):
    omega = params["omega"]; Tc = params["Tc"]
    term1 = 0.37464 + 1.54226 * omega - 0.26992 * (omega ** 2)
    term2 = 1 - cp.sqrt(T / Tc)
    return (1 + term1 * term2) ** 2

def PR_Pressure(rho, T, alpT, params):
    a = params["a"]; b = params["b"]; R = params["R"]
    Pterm1 = rho * R * T / (1 - b * rho)
    Pterm2 = a * alpT * (rho ** 2) / (1 + 2 * b * rho - (b ** 2) * (rho ** 2))
    return Pterm1 - Pterm2

def compute_psi(newP, rho, G, cs2):
    """ Yuan & Schaefer Effective Mass (psi) psi = sqrt( 2(P_eos - rho*cs^2) / (G*cs^2) )    """
    num = 2 * (newP - rho * cs2)
    den = G * cs2
    arg = num / den
    arg = cp.maximum(arg, 1e-12) 
    #rho1 = rho0 * (1 - cp.exp(-rho / rho0))
    return cp.sqrt(arg)

# Shan-chen force

In [ ]:

def shan_chen_force(psi, G, c, geom_shape, beta=beta):
    """
    [Gong&Cheng, 2012, Computers&Fluids] Beta-corrected Shan-Chen Force
    Beta: 0.886 (Shan-Chen), 0.55 (vdW), 1.16 (P-R)
    """
    nz, ny, nx = geom_shape
    Fx = cp.zeros((nz, ny, nx), dtype='f8')
    Fy = cp.zeros((nz, ny, nx), dtype='f8')
    Fz = cp.zeros((nz, ny, nx), dtype='f8')
    
    psi_sq = psi ** 2

    #w_force = cp.array([0] + [1.0/6.0]*6 + [1.0/12.0]*12, dtype='f8')
    w_force = cp.array([0] + [1.0/18.0]*6 + [1.0/36.0]*12, dtype='f8')
    for i in range(1, 19): 
        cx, cy, cz = int(c[i,0]), int(c[i,1]), int(c[i,2])
        
        # Shift
        psi_neighbor = cp.roll(psi, shift=(-cz, -cy, -cx), axis=(0, 1, 2))
        psi_sq_neighbor = cp.roll(psi_sq, shift=(-cz, -cy, -cx), axis=(0, 1, 2))
        
        # bracket = beta * psi * psi' + 0.5 * (1-beta) * psi'^2
        term_interactive = beta * psi * psi_neighbor
        term_correction  = 0.5 * (1.0 - beta) * psi_sq_neighbor
        bracket_sum = term_interactive + term_correction

        weight = w_force[i]
        Fx += weight * bracket_sum * cx
        Fy += weight * bracket_sum * cy
        Fz += weight * bracket_sum * cz

    Fx *= -G
    Fy *= -G
    Fz *= -G
    
    return cp.stack((Fx, Fy, Fz), axis=0) # Ftotal 리턴


# Equilibrium (f, g, momentumP)

In [ ]:

def compute_momentum(f, geom, params, P):
    c = geom["c"]; Q = params["Q"]; S = params["S"]
    P[:] = 0
    for i in range(Q):
        P[0] += f[i] * c[i, 0] * S
        P[1] += f[i] * c[i, 1] * S
        P[2] += f[i] * c[i, 2] * S
    return P

def f_equilibrium(rho, u, geom, params):
    c = geom["c"]; w = geom["w"]
    S = params["S"]; cs2 = params["cs2"]; cs4 = params["cs4"]
    S_u = u * S
    u_sq_sum = u[0]**2 + u[1]**2 + u[2]**2
    c_expanded = c[:, :, cp.newaxis, cp.newaxis, cp.newaxis]  # (Q,3,1,1,1)
    cu = cp.sum(c_expanded * S_u, axis=1)                     # (Q,nz,ny,nx)
    return rho * w[:, None, None, None] * (1 + cu / cs2 + (cu**2) / (2 * cs4) - u_sq_sum / (2 * cs2))

def g_equilibrium(T, u, geom, params):
    c = geom["c"]; w = geom["w"]
    S = params["S"]; cs2 = params["cs2"]; cs4 = params["cs4"]
    S_u = u * S
    u_sq_sum = u[0]**2 + u[1]**2 + u[2]**2
    c_expanded = c[:, :, cp.newaxis, cp.newaxis, cp.newaxis]
    cu = cp.sum(c_expanded * S_u, axis=1)
    #g_eq = w[:, None, None, None] * T * (1 + cu / cs2 + (cu**2) / cs2 - u_sq_sum / (2 * cs2)) # quadratic
    g_eq_only_advection = w[:, None, None, None] * T * (1 + cu / cs2) # linear
    return g_eq_only_advection

# Heat source (Latent heat)

In [ ]:
### for work ###
def div_u_from_u(u):
    dux_dx = (cp.roll(u[0], -1, axis=2) - cp.roll(u[0], 1, axis=2)) * 0.5
    duy_dy = (cp.roll(u[1], -1, axis=1) - cp.roll(u[1], 1, axis=1)) * 0.5
    duz_dz = (cp.roll(u[2], -1, axis=0) - cp.roll(u[2], 1, axis=0)) * 0.5
    return dux_dx + duy_dy + duz_dz

def dpdT_rho_PR(rho, T, alpT, params):
    a = params["a"]; b = params["b"]; R = params["R"]
    omega = params["omega"]; Tc = params["Tc"]
    eps = params.get("eps", 1e-10)
    m = 0.37464 + 1.54226*omega - 0.26992*(omega**2)
    sqrt_alp = cp.sqrt(alpT)
    dalp_dT = - (m * sqrt_alp) / (cp.sqrt(T*Tc) + eps)
    denom = 1 + 2 * b * rho - (b ** 2) * (rho ** 2)
    term1 = (rho * R) / (1 - b*rho + eps)
    term2 = (a * rho**2 / (denom + eps)) * dalp_dT
    return term1 - term2

# COllision (EDM or Guo's forcing for latent heat)

In [ ]:
def collide_with_source(f, g, f_eq_mom, f_eq_edm, f_eq_phy, g_eq,
                        rho, T, u_phy, Ftotal, alpT, geom, params):
    dt = params["dt"]; eps = params.get("eps", 1e-10)
    method = params.get("forcing_method", None)
    phi = cp.zeros_like(T)
    # --- 1. Thermal Source (Latent + Work) ---
    # div(rho u)
    rhoUx = rho * u_phy[0]; rhoUy = rho * u_phy[1]; rhoUz = rho * u_phy[2]
    drhoUx_dx = (cp.roll(rhoUx, -1, axis=2) - cp.roll(rhoUx, 1, axis=2)) * 0.5
    drhoUy_dy = (cp.roll(rhoUy, -1, axis=1) - cp.roll(rhoUy, 1, axis=1)) * 0.5
    drhoUz_dz = (cp.roll(rhoUz, -1, axis=0) - cp.roll(rhoUz, 1, axis=0)) * 0.5
    div_rhoU = drhoUx_dx + drhoUy_dy + drhoUz_dz
    divu = div_u_from_u(u_phy)
    total_source = 0.0
    if params.get("use_work", False):
        dpdt = dpdT_rho_PR(rho, T, alpT, params)
        Cv = params.get("Cv")
        if params.get("use_dampWork", False):
            phi = T * (1.0 - dpdt / (rho * Cv + eps))
            source_work = phi * divu
        else:
            source_work = - T * dpdt * divu / (rho * Cv + eps)
        total_source += source_work
        #print(cp.min(total_source), cp.max(total_source), "total_source before latent")

     # if params.get("use_latent", False):
     #    total_source -= params["L_lb"] * div_rhoU * 0.001

    # ice 승화포인트 삼중점
    T_triple_C = -56.6
    T_triple_K = T_triple_C + 273.15
    
    if params.get("use_latent", False):
        mask_solid = (state["T"] * C_T) < T_triple_K 
        # 3. 잠열 소스 계산
        latent_source = params["L_lb"] * mask_solid 
        total_source += latent_source * 0.07 # relaxation factor
        #print(cp.min(total_source), cp.max(total_source), "total_source after latent")
    source_term_g = geom["w"][:, None, None, None] * total_source * dt
    
    # --- 2. Momentum Collision (EDM vs Guo) ---
    f_c = cp.zeros_like(f)

    if method == "EDM":
        # [EDM] f_new = f - (f - f_eq_mom)/tau + (f_eq_edm - f_eq_mom) # Kupershtokh et al.
        f_c = f - (f - f_eq_mom) * (dt / params["tau_f"]) + (f_eq_edm - f_eq_mom)
        
    elif method == "Guo": # (Wang et al., 2019)
        # [Guo-Zheng-Shi] f_new = f - (f - f_eq_phy)/tau + Source_Guo
        # c_i dot F
        c = geom["c"] # (19, 3)
        cx, cy, cz = c[:, 0], c[:, 1], c[:, 2]
        
        c_dot_F = (cx[:, None, None, None] * Ftotal[0] + 
                   cy[:, None, None, None] * Ftotal[1] + 
                   cz[:, None, None, None] * Ftotal[2])
        
        # u dot F (u_phy 사용)
        u_dot_F = (u_phy[0] * Ftotal[0] + u_phy[1] * Ftotal[1] + u_phy[2] * Ftotal[2])
        
        # c dot u (u_phy 사용)
        c_exp = c[:, :, None, None, None] # (19, 3, 1, 1, 1)
        u_exp = u_phy[None, :, :, :, :]   # (1, 3, nz, ny, nx) * S 
        S_u = u_phy * params["S"]
        c_dot_u = cp.sum(c_exp * S_u, axis=1) # (19, nz, ny, nx)

        w_ = geom["w"][:, None, None, None]
        cs2 = params["cs2"]
        coeff = (1.0 - 0.5 * dt / params["tau_f"])
        
        t1 = c_dot_F / cs2 # (c.F)/cs2
        t2 = (c_dot_u * c_dot_F) / (cs2 ** 2) # (c.u)(c.F)/cs4
        t3 = u_dot_F / cs2 # (u.F)/cs2
        
        Guo_force = w_ * coeff * (t1 + t2 - t3) * dt
        
        f_c = f - (f - f_eq_phy) * (dt / params["tau_f"]) + Guo_force

    g_c = g - (g - g_eq) * (dt / params["tau_g"]) + source_term_g

    return f_c, g_c, phi, mask_solid


# Streaming

In [ ]:

def stream_step(f_c, g_c, f_stream, g_stream, geom):
    c = geom["c"]
    Q = f_c.shape[0]
    for k in range(Q):
        dz, dy, dx = int(c[k,2]), int(c[k,1]), int(c[k,0])

        f_stream[k] = cp.roll(f_c[k], shift=(dz, dy, dx), axis=(0,1,2)) # 일단 전체 밀기 반대편값이 튀어나오는거 밑에서 조정
        g_stream[k] = cp.roll(g_c[k], shift=(dz, dy, dx), axis=(0,1,2))
        
        if dx > 0: 
            f_stream[k, :, :, :dx] = 0
            g_stream[k, :, :, :dx] = 0
        elif dx < 0: 
            f_stream[k, :, :, dx:] = 0
            g_stream[k, :, :, dx:] = 0
            
        if dy > 0: 
            f_stream[k, :, :dy, :] = 0
            g_stream[k, :, :dy, :] = 0
        elif dy < 0: 
            f_stream[k, :, dy:, :] = 0
            g_stream[k, :, dy:, :] = 0
            
        if dz > 0: 
            f_stream[k, :dz, :, :] = 0
            g_stream[k, :dz, :, :] = 0
        elif dz < 0: 
            f_stream[k, dz:, :, :] = 0
            g_stream[k, dz:, :, :] = 0
            
    return f_stream, g_stream


# Boundary conditions

In [ ]:
def apply_bc(f_c, g_c, f_stream, g_stream, geom, params, state):
    Q = params["Q"]; w = geom["w"]; ai = geom["ai"]
    top, bottom = geom["top_isn_indices"], geom["bottom_isn_indices"]
    left, right = geom["left_isn_indices"], geom["right_isn_indices"]
    front, back = geom["front_isn_indices"], geom["back_isn_indices"]
    nozzles, particles = geom["nozzles"], geom["particles"]
    corner = geom["corner_isn_indices"]
    u = state["u"]
    rho = state["rho"]
    high_rho = params["high_rho"] 
    low_rho = params["low_rho"]  
    T_inlet = params["T_inlet"]
    T_ambient = params.get("backT", 1.0) 

    u_inlet_full = cp.zeros_like(u)
    # u_inlet_full[0] = inlet_vel 추가가능?
    rho_inlet_full = cp.full_like(rho, high_rho)
    f_eq_inlet = f_equilibrium(rho_inlet_full, u_inlet_full, geom, params)

    u_open = u.copy()
    rho_open = cp.full_like(rho, low_rho)
    f_eq_open = f_equilibrium(rho_open, u_open, geom, params)

    for l in range(Q):
        open_idx = [top, bottom, front, back, corner]
        for idx in open_idx:
            f_stream[l][idx] = f_eq_open[l][idx]
            g_stream[l][idx] = w[l] * T_inlet

        f_stream[l][left] = f_eq_inlet[l][left]
        g_stream[l][left] = w[l] * T_inlet
        #f_stream[l][left] = f_c[ai[l]][left]
        f_stream[l][nozzles] = f_c[ai[l]][nozzles]
        f_stream[l][particles] = f_c[ai[l]][particles]
        
        g_stream[l][nozzles] = g_c[ai[l]][nozzles]
        g_stream[l][particles] = g_c[ai[l]][particles]

        f_stream[l, :, :, -1] = f_stream[l, :, :, -2]
        g_stream[l, :, :, -1] = g_stream[l, :, :, -2]

    return f_stream, g_stream

# Main Loop

In [ ]:
if params["forcing_method"] == "Guo" and params["G"] > -0.5:
    print("Warning: Guo forcing is accurate but sensitive. Ensure G is strong (-1.0).")
sigma = []; vel_exact_ = []; mass_evolution = []
def vis_f_func(T, vis_f, params):
    T_ref = params["Tc"]
    #vis_ref = params["vis_f"]
    return vis_f * (T / Tc) ** 0.7# Power law Sutherland나 Chapman-Enskog
def P_sat_antoine(T_real):  # T in K Antoine 
            return 10 ** (6.81228 - 1301.679 / (T_real - 3.494)) * 1e5  # Pa
solid_history = cp.zeros(T_result.shape, dtype=cp.bool_)

for step in range(1, steps + 1):
    total_mass = 0
    # 1. Macroscopic Variables
    rho = cp.sum(f, axis=0)
    T_result = cp.sum(g, axis=0) 
    total_mass = cp.sum(rho)
    mass_evolution.append(total_mass)
    # 2. Momentum (P) Calculation
    P[0] = 0; P[1] = 0; P[2] = 0
    Px = cp.sum(f * c[:,0][:,None,None,None], axis=0)
    Py = cp.sum(f * c[:,1][:,None,None,None], axis=0)
    Pz = cp.sum(f * c[:,2][:,None,None,None], axis=0)

    #vis_f = vis_f_func(T_result, vis_f, params)
    #tau_f = cp.clip(3 * vis_f + 0.5, 0.51, 2.0)
    #tau_f = tau_f[cp.newaxis, :, :, :] 

    # 3. Compute Forces
    # EOS Pressure & Psi
    alpT = alphaT(T_result, params)
    newP = PR_Pressure(rho, T_result, alpT, params)
    psi = compute_psi(newP, rho, params["G"], params["cs2"])
    if step == 2: 
        initial_low_P = cp.min(newP)
        initial_high_P = cp.max(newP)
        print("minP", initial_low_P* C_P /1e6, "MPa", "maxP", initial_high_P * C_P /1e6, "MPa")
        plt.title('XZ: Initial newP')
        plt.imshow(cp.asnumpy(newP[:,ny//2,:]*C_P/1e6), cmap='jet', origin='lower')
        plt.colorbar()
        plt.show()

    # Shan-Chen Force
    F_total = shan_chen_force(psi, params["G"], geom["c"], (nz, ny, nx), beta=params["beta"])
    
    # Force Ramping 
    if step <= 500:
        F_total *= (step / 500.0)
        #high_rho_current = high_rho * min(low_rho, step / 500.0)
        
    # Velocity Definitions 
    u_mom = cp.zeros_like(u)
    u_mom[0] = Px / rho # momentum velocity (f)
    u_mom[1] = Py / rho
    u_mom[2] = Pz / rho
    
    max_vel = 0.2
    v_mag = cp.sqrt(u_mom[0]**2 + u_mom[1]**2 + u_mom[2]**2)
    mask = v_mag > max_vel
    if cp.any(mask):
        factor = max_vel / (v_mag + 1e-12)
        factor = cp.where(mask, factor, 1.0)
        u_mom *= factor

    u_phy = cp.zeros_like(u) # real velocity (physical velocity)
    u_phy[0] = u_mom[0] + 0.5 * F_total[0] * dt / rho
    u_phy[1] = u_mom[1] + 0.5 * F_total[1] * dt / rho
    u_phy[2] = u_mom[2] + 0.5 * F_total[2] * dt / rho

    u_edm = cp.zeros_like(u)
    u_edm[0] = u_mom[0] + F_total[0] * dt / rho
    u_edm[1] = u_mom[1] + F_total[1] * dt / rho
    u_edm[2] = u_mom[2] + F_total[2] * dt / rho
    
    state["u"] = u_phy 
    state["rho"] = rho
    state["T"] = T_result
    state["newP"] = newP
    
    # 5. Collision (Guo Forcing Correct Implementation)
    f_eq_mom= f_equilibrium(rho, u_mom, geom, params)
    f_eq_edm= f_equilibrium(rho, u_edm, geom, params)
    f_eq_phy= f_equilibrium(rho, u_phy, geom, params)
    
    g_eq = g_equilibrium(T_result, u_phy, geom, params)
    
    f_c, g_c, phi, mask_solid = collide_with_source(f, g, f_eq_mom, f_eq_edm, f_eq_phy, g_eq,
                        rho, T_result, u_phy, F_total, alpT, geom, params)
    # 6. Streaming
    # for i in range(Q): # oeriodic condition
    #     sx, sy, sz = int(c[i,0]), int(c[i,1]), int(c[i,2])
    #     f[i] = cp.roll(f_c[i], shift=(sz, sy, sx), axis=(0, 1, 2))
    #     g[i] = cp.roll(g_c[i], shift=(sz, sy, sx), axis=(0, 1, 2))
        
    f_stream, g_stream = stream_step(f_c, g_c, f_stream, g_stream, geom)
    f, g = apply_bc(f_c, g_c, f_stream, g_stream, geom, params, state)
    ### solid 포함 or not
    #sublimation_mask = (T_result * C_T - 273.13 > -50.0)
    #solid_history = cp.logical_and(solid_history, ~sublimation_mask)
    
    mask_solid_now = (T_result * C_T - 273.13 < -56.6) & (newP * C_P / 1e6 > 0.5)
    solid_history = cp.logical_or(solid_history, mask_solid_now)
    
    P_sat_T = P_sat_antoine(T_result)  # temperature dependent satrutation pressure
    index = cp.ones_like(T_result)  # gas
    #index = cp.where(nozzle_mask == 1, 4, index)
    index = cp.where((T_result < Tc) & (newP > P_sat_T), 2, index)  # liquid
    index = cp.where((T_result > Tc) & (newP > Pc), 3, index)        # supercritical
    index = cp.where(solid_history, 4, index) 
    # ======== GIF (with quiver overlay) ======== #
    if make_gif and (step % gif_every == 0 or step == 1):
        if gif_field == "rho":
            field3d = state["rho"] * C_rho
            cmap_name = "Blues"
        elif gif_field == "T":
            field3d = state["T"] * C_T - 273.13
            cmap_name = "turbo"
        elif gif_field == "psi":
            field3d = psi * C_P
            cmap_name = "twilight"
        elif gif_field == "vel":
            u_ = state["u"] * C_u
            field3d = cp.sqrt(u_[0]**2 + u_[1]**2 + u_[2]**2)
            cmap_name = "magma"
        elif gif_field == "index":
            field3d = index
            cmap_name = ListedColormap(['whitesmoke', 'mediumblue', 'skyblue', 'black']) #"twilight"
        else:
            raise ValueError("gif_field must be one of: rho, T, psi, vel")
    
        ax, idx = gif_slice
        field2d = _get_slice(cp.asnumpy(field3d), ax, idx)
        field2d = field2d[::-1, :]  
        X2d = cp.asnumpy(geom["XXX"]); Z2d = cp.asnumpy(geom["ZZZ"])
        U2d = cp.asnumpy(state["u"][0, :, ny//2, :]); W2d = cp.asnumpy(state["u"][2, :, ny//2, :])
        
        if field2d.shape != U2d.shape:
            if field2d.T.shape == U2d.shape:
                field2d = field2d.T
            else:
                print("WARNING shape mismatch:", field2d.shape, U2d.shape)
        if vmin is None or vmax is None:
            _vmin = float(np.nanmin(field2d))
            _vmax = float(np.nanmax(field2d))
        else:
            _vmin, _vmax = vmin, vmax

        frame = make_overlay_frame(
            field2d, X2d, Z2d, U2d, W2d,
            vmin=_vmin, vmax=_vmax, cmap_name=cmap_name,
            stride=quiver_stride,
            qscale=quiver_scale,
            qcolor=quiver_color,
            qalpha=quiver_alpha, q_color_field=(state["T"][:, ny//2, :].get())*C_T-273.13, q_cmap="coolwarm", q_clim=(-30,150)
        )
        #frame = add_colorbar_right(frame, vmin=_vmin, vmax=_vmax, cmap_name=cmap_name, cb_w=45, pad=6)
        frame = add_step_bar(frame, step, C_time, bar_h=25)
        img = Image.fromarray(frame)
        img = img.resize((img.width * scale, img.height * scale), Image.NEAREST)
        frame = np.array(img)
        frames.append(frame)

    # ======== VISUALIZATION ======== #
    if step == 2 or step % 50 == 0:
        print(step, "th step") 
        u = state["u"]
        rho = state["rho"]
        newP = state["newP"]
        T = state["T"]

        volume_nodes = cp.sum(rho > rhoc)
        #specificVolume = 1 / rho[nz//2, ny//2, nx//2]
        
        vel = cp.sqrt(u[0]**2 + u[1]**2 + u[2]**2)
        vel_exact_.append(cp.average(vel))
        max_rho = cp.max(rho).item()
        min_rho = cp.min(rho).item()
        min_denom = cp.min(1.0 - params["b"] * rho).item()
        print(f"Step {step}: Max Rho={max_rho:.4f}, Min(1-b*rho)={min_denom:.6f}")
        LiguidSpecificVolume = 1.0 / max_rho
        GasSpecificVolume = 1.0 / min_rho
        print(f" v_liq: {LiguidSpecificVolume:.4f} | v_gas: {GasSpecificVolume:.4f}")
        deltaP = newP[nz//2, ny//2 , wall_start] - newP[nz//2, ny//2 , wall_end-1]
        deltaT = T[nz//2, ny//2 , wall_start] - T[nz//2, ny//2 , wall_end-1]
        JT = (deltaT/deltaP)
        C_JT_unit = C_T / C_P * 1e6
        print(f'JT: {JT} lb')
        print(f'JT: {JT * C_JT_unit} K/MPa')
        print(f'current mean T: {cp.mean(T) * C_T} K ')
        print(f'current min T: {cp.min(T) * C_T} K ')
        print(f'initial T: {T_inlet * C_T} K ')
        print(f'meanP: {cp.mean(newP) * C_P/ 1e6} MPa')
        print(f'JT in NIST: {2.2} K/MPa')
        print("Pin-Pout:", float(deltaP * C_P))
        print("Tin-Tout:", float(deltaT))
        
        print("max |u|:", float(cp.max(vel)))
        print("max/min rho:", float(cp.max(rho)), float(cp.min(rho)))
        print("max/min psi:", float(cp.max(psi)), float(cp.min(psi)))

        maxMa = cp.max(cp.sqrt(u[0]**2+u[1]**2+u[2]**2))/cs
        print("maxMa:", float(maxMa))

        Ma_distribution = (cp.sqrt(u[0]**2+u[1]**2+u[2]**2))/cs
        plt.title('Mach at XZ plane')
        plt.imshow(cp.asnumpy(Ma_distribution[:,ny//2,:]), cmap='magma', origin='lower')
        plt.clim(0,1.0)
        plt.colorbar(orientation='horizontal')
        plt.show()
        
        p0 = state["rho"]*cs2
        print("newP range:", float(cp.min(newP)), float(cp.max(newP)))
        print("rho*cs2 range:", float(cp.min(p0)), float(cp.max(p0)))
        print("u_max:", float(cp.max(vel)))
        F = F_total
        Fmag = cp.sqrt(F[0]**2 + F[1]**2 + F[2]**2)
        print("Ftotal_max:", float(cp.max(Fmag)), "Fwet_max:", float(cp.max(cp.sqrt(Fwet[0]**2+Fwet[1]**2+Fwet[2]**2))))

        print("===== Real Values =====")
        print('max rho: ', cp.max(rho) * C_rho, 'kg/m^3', f"| rhoc: 467.6 kg/m^3") # 2.66 mu/lu^3 : 322 kg/m^3
        print('min rho: ', cp.min(rho) * C_rho, 'kg/m^3', f"| rhoI: {high_rho * C_rho}, {low_rho * C_rho} kg/m^3") # 2.66 mu/lu^3 : 322 kg/m^3
        
        print('max T: ', cp.max(T) * C_T - 273.15, 'Celcius', f"| Tc: 30.98 degree celcius") # 2.66 mu/lu^3 : 322 kg/m^3
        print('min T: ', cp.min(T) * C_T -273.15, 'Celcius', f"| TI: {T_inlet * C_T - 273.13} degree celcius") # 2.66 mu/lu^3 : 322 kg/m^3
        
        print('max P: ', cp.max(newP) * C_P / 1e6, 'MPa', f"| Pc: 7.38 MPa") # 2.66 mu/lu^3 : 322 kg/m^3
        print('min P: ', cp.min(newP) * C_P / 1e6, 'MPa', f"| PI: {initial_low_P / 1e6} MPa") # 2.66 mu/lu^3 : 322 kg/m^3
        print('rho/rhoc: ', cp.max(rho)/rhoc)
        print("=======================")
        vmin_x, vmax_x = np.percentile(u, [5, 95])
        vmin_y, vmax_y = np.percentile(u, [30, 70])
        vmin_z, vmax_z = np.percentile(u, [30, 70])
        vmin_T, vmax_T = np.percentile(T, [5, 95])
        u_visual = u.copy()
        idx_z, idx_y, idx_x = nozzles 
        b_z, b_y, b_x = bottom
        t_z, t_y, t_x = top
        u_visual[:, idx_z, idx_y, idx_x] = cp.nan
        u_visual[:, b_z, b_y, b_x] = cp.nan
        u_visual[:, t_z, t_y, t_x] = cp.nan
        current_cmap = plt.cm.jet
        current_cmap.set_bad(color='black')
        
        plt.figure(figsize=(10,8))
        plt.title('streamline at {}'.format(step))
        X_slice = cp.asnumpy(geom["XXX"])
        Z_slice = cp.asnumpy(geom["ZZZ"])
        U_slice = cp.asnumpy(u_visual[0, :, ny//2, :])
        W_slice = cp.asnumpy(u_visual[2, :, ny//2, :])
        T_visual = T.copy()
        T_visual -= T_init
        T_visual *= Tc # convert to Kelvin
        #T_visual -= 273.15 # convert degree Celcius
        limit = cp.max(cp.abs(T_visual[:, ny//2, :]))

        cmap = ListedColormap(['whitesmoke', 'steelblue', 'skyblue', 'black'])
        plt.imshow(index[:,ny//2,:].get(), cmap=cmap, vmin=1, vmax=4)
        plt.colorbar(ticks=[1,2,3,4], label=['gas','liquid','SC','ice'], orientation='horizontal',fraction=0.07)
        plt.quiver(X_slice, Z_slice, U_slice, W_slice, cp.asnumpy(T[:, ny//2, :])*C_T-273.13, cmap = 'coolwarm', clim = (-60, 300))
        plt.colorbar(label = f'dT (degree) (minT: {cp.min(T_visual[:, ny//2, :]): g} degree)', orientation='horizontal',fraction=0.07)
        plt.tight_layout()
        plt.show()
        
        fig, axs = plt.subplots(2, 2)
        im1 = axs[0, 0].imshow(cp.asnumpy(u_visual[2,:,ny//2,:] * C_u), cmap=current_cmap, origin='lower')
        axs[0, 0].set_title('XZ: uz at {}'.format(step))
        plt.colorbar(im1, ax = axs[0, 0])
        #im1.set_clim(vmin_z, vmax_z)
        im2 = axs[0, 1].imshow(cp.asnumpy(u_visual[1,:,ny//2,:] * C_u), cmap=current_cmap, origin='lower')
        axs[0, 1].set_title('XZ: uy at {}'.format(step))
        #im2.set_clim(vmin_y, vmax_y)
        plt.colorbar(im2, ax = axs[0, 1])
        im3 = axs[1, 0].imshow(cp.asnumpy(u_visual[0,:,ny//2,:] * C_u), cmap=current_cmap, origin='lower')
        axs[1, 0].set_title('XZ: ux at {}'.format(step))
        #im3.set_clim(vmin_x, vmax_x)
        plt.colorbar(im3, ax = axs[1, 0])
        im4 = axs[1, 1].imshow(cp.asnumpy(T[:,ny//2,:] * C_T -273.13), cmap=current_cmap, origin='lower')
        axs[1, 1].set_title('XZ: Temperature')
        plt.colorbar(im4, ax = axs[1, 1])
        #im4.set_clim(0, 200)
        fig.tight_layout()
        plt.show()
          
        fig, axs = plt.subplots(2, 2)
        im1 = axs[0, 0].imshow(cp.asnumpy(u[2,:,:,nx//2] * C_u), cmap=current_cmap, origin='lower')
        axs[0, 0].set_title('YZ: uz at {}'.format(step))
        plt.colorbar(im1, ax = axs[0, 0])
       # im1.set_clim(vmin_z, vmax_z)
        im2 = axs[0, 1].imshow(cp.asnumpy(u[1,:,:,nx//2] * C_u), cmap=current_cmap, origin='lower')
        axs[0, 1].set_title('YZ: uy at {}'.format(step))
        plt.colorbar(im2, ax = axs[0, 1])
       # im2.set_clim(vmin_y, vmax_y)
        im3 = axs[1, 0].imshow(cp.asnumpy(u[0,:,:,nx//2] * C_u), cmap=current_cmap, origin='lower')
        axs[1, 0].set_title('YZ: ux at {}'.format(step))
        plt.colorbar(im3, ax = axs[1, 0])
       # im3.set_clim(vmin_x, vmax_x)
        im4 = axs[1, 1].imshow(cp.asnumpy(T[:,:,nx//2] * C_T -273.13), cmap=current_cmap, origin='lower')
        axs[1, 1].set_title('YZ: Temperature')
        plt.colorbar(im4, ax = axs[1, 1])
       # im4.set_clim(vmin_T, vmax_T)
        fig.tight_layout()
        plt.show()

        plt.title('XZ: newP')
        plt.imshow(cp.asnumpy(newP[:,ny//2,:]*C_P/1e6), cmap='jet', origin='lower')
        #plt.clim(1.460512821,injectionRho)
        plt.colorbar()
        plt.show()
        plt.title('XZ: rho_result')
        plt.imshow(cp.asnumpy(rho[:,ny//2,:]*C_rho), cmap='jet', origin='lower')
        plt.clim(0, rhoc * C_rho)
        plt.colorbar()
        plt.show()
        
        plt.title('newP at XZ plane')
        plt.plot(cp.asnumpy(newP[:,ny//2,:]*C_P/1e6),'-')
        plt.show()
        plt.title('rho at XZ plane')
        plt.plot(cp.asnumpy(rho[:,ny//2,:]*C_rho),'-')
        plt.show()
        plt.title('T at XZ plane')
        plt.plot(cp.asnumpy(T[:,ny//2,:]*C_T -273.13),'-')
        plt.show()
        plt.title('sigma evolution')
        plt.plot(cp.asnumpy(cp.array(sigma)),'*--')
        plt.show()
        plt.title('mass evolution')
        plt.plot(cp.asnumpy(cp.array(mass_evolution)),'*--')
        plt.show()
        
        g_sum = cp.sum(g, axis=0)   # 너 정의가 g가 internal energy*rho 라면
        print("g_sum min/max:", float(cp.min(g_sum)), float(cp.max(g_sum)))
        print("rho min/max:", float(cp.min(rho_result)), float(cp.max(rho_result)))
        print("T min/max:", float(cp.min(T_result)), float(cp.max(T_result)))
        idx = cp.unravel_index(cp.argmin(T_result), T_result.shape)
        print("argmin T idx:", [int(i) for i in idx])
        print("rho at Tmin:", float(rho_result[idx]), "g_sum at Tmin:", float(g_sum[idx]))
        divu = (u[0,1:-1,1:-1,2:] - u[0,1:-1,1:-1,:-2] +
                u[1,1:-1,2:,1:-1] - u[1,1:-1,:-2,1:-1] +
                u[2,2:,1:-1,1:-1] - u[2,:-2,1:-1,1:-1]) / 2
        print("divu min/max:", float(cp.min(divu)), float(cp.max(divu)))
        print("T_real(K) min/max:", float(cp.min(T_result)*C_T), float(cp.max(T_result)*C_T))
        print("T_real(C) min/max:", float(cp.min(T_result)*C_T - 273.15), float(cp.max(T_result)*C_T - 273.15))
        #if step % 200 == 0:
        if step  == 2: # initial
            VTKConverter(step, nx, ny, nz, u * C_u, T * C_T - 273.13, rho * C_rho, newP * C_P/1e6, index)
if make_gif: 
    iio.imwrite(gif_path, frames, duration=0.1, loop=0)
    print("Saved GIF:", gif_path)
            

# DONE

In [ ]:
if make_gif:     # duration: 한 프레임이 몇 초인지 (예: 0.08 = 12.5fps)
    iio.imwrite(gif_path, frames, duration=0.1, loop=0)
    print("Saved GIF:", gif_path)

vol_1d = 1/cp.reshape(rho,-1)
newP_1d = cp.reshape(newP,-1)

plt.plot(vol_1d.get(), newP_1d.get())

In [ ]:
deltaP = newP[nz//2, ny//2 , 1] - newP[nz//2, ny//2 , wall_end-1]
deltaT = T[nz//2, ny//2 , 1] - T[nz//2, ny//2 , wall_end-1]
JT = (deltaT/deltaP)
C_JT_unit = C_T / C_P * 1e6
print(f'JT: {JT} lb')
print(f'JT: {JT * C_JT_unit} K/MPa')
print(f'current mean T: {cp.mean(T) * C_T} K ')
print(f'current min T: {cp.min(T) * C_T} K ')
print(f'initial T: {T_inlet * C_T} K ')
print(f'meanP: {cp.mean(newP) * C_P/ 1e6} MPa')
print(f'JT in NIST: {2.2} K/MPa')
print("Pin-Pout:", float(deltaP * C_P))
print("Tin-Tout:", float(deltaT))

In [ ]:

# for i in range(nz):
#     for j in range(ny):
#         for k in range(nx):# 1: gas 2: fluid 3: Sc 4: solid
#             if T[i, j, k] < Tc and newP[i, j, k] < Pc: index[i, j, k] = 1
#             if T[i, j, k] < Tc and newP[i, j, k] > Pc: index[i, j, k] = 2
#             if T[i, j, k] > Tc and newP[i, j, k] < Pc: index[i, j, k] = 1
#             if T[i, j, k] > Tc and newP[i, j, k] > Pc: index[i, j, k] = 3
# CO2 Antoine (간단 근사)

# index = cp.ones_like(T)  # 기본 1: gas
# # T < Tc, P > Pc : liquid
# index = cp.where((T < Tc) & (newP > Pc), 2, index)
# # T > Tc, P > Pc : supercritical  
# index = cp.where((T > Tc) & (newP > Pc), 3, index)
# # T > Tc, P < Pc : gas (이미 1이니까 생략 가능) 


In [ ]:
plt.figure(figsize=(8,8))
plt.title('streamline at {}'.format(step))
X_slice = cp.asnumpy(geom["XXX"])
Z_slice = cp.asnumpy(geom["ZZZ"])
U_slice = cp.asnumpy(u_visual[0, :, ny//2, :])
W_slice = cp.asnumpy(u_visual[2, :, ny//2, :])
T_visual = T.copy()
T_visual -= T_init
T_visual *= 1.77e+04 # convert to Kelvin
T_visual -= 273.15 # convert degree Celcius
plt.quiver(X_slice, Z_slice, U_slice, W_slice, cp.asnumpy(T_visual[:, ny//2, :]),  cmap = 'coolwarm' )
plt.colorbar(label = f'T (degree) (minT: {cp.min(T_visual[:, ny//2, :]): g} degree)')
print(cp.min(T_visual[:, ny//2, :]))
print(0.026 * 1.77e4 - 273.15)

In [ ]:
v = 1.0 / rho_result
print("min(v-b):", cp.min(v - b))
print("max rho:", cp.max(rho_result), " 1/b:", 1.0/b)
print("min T:", cp.min(T_result))
p_ideal = cs2 * rho_result
print("ideal (MPa):", float(cp.min(p_ideal)*C_P/1e6), float(cp.max(p_ideal)*C_P/1e6))

# 너의 newP가 EOS 전체인지/비이상부인지 확인하려면:
p_total = newP + p_ideal
print("total (MPa):", float(cp.min(p_total)*C_P/1e6), float(cp.max(p_total)*C_P/1e6))


In [ ]:
rho_ref = float(cp.mean(rho_result[:, :, :wall_start]))  # inlet 쪽 평균 같은 걸로
p_ref_lu = (cs**2) * rho_ref
P_ref_real = 13e6  # Pa (또는 8e6)

C_P_new = P_ref_real / p_ref_lu
print("p_ref_lu:", p_ref_lu, "C_P_new:", C_P_new)
print(C_P/C_P_new)

p_ideal = cs2 * rho_result
p_total = newP + p_ideal

print("total MPa (new scale):",
      float(cp.min(p_total)*C_P_new/1e6),
      float(cp.max(p_total)*C_P_new/1e6))


In [ ]:
g_sum = cp.sum(g, axis=0)   # 너 정의가 g가 internal energy*rho 라면
print("g_sum min/max:", float(cp.min(g_sum)), float(cp.max(g_sum)))
print("rho min/max:", float(cp.min(rho_result)), float(cp.max(rho_result)))
print("T min/max:", float(cp.min(T_result)), float(cp.max(T_result)))
idx = cp.unravel_index(cp.argmin(T_result), T_result.shape)
print("argmin T idx:", [int(i) for i in idx])
print("rho at Tmin:", float(rho_result[idx]), "g_sum at Tmin:", float(g_sum[idx]))
divu = (u[0,1:-1,1:-1,2:] - u[0,1:-1,1:-1,:-2] +
        u[1,1:-1,2:,1:-1] - u[1,1:-1,:-2,1:-1] +
        u[2,2:,1:-1,1:-1] - u[2,:-2,1:-1,1:-1]) / 2
print("divu min/max:", float(cp.min(divu)), float(cp.max(divu)))
print("T_real(K) min/max:", float(cp.min(T_result)*C_T), float(cp.max(T_result)*C_T))
print("T_real(C) min/max:", float(cp.min(T_result)*C_T - 273.15), float(cp.max(T_result)*C_T - 273.15))


In [ ]:
print("phi min/max:", cp.min(phi), cp.max(phi))
idx = cp.unravel_index(cp.argmin(T_result), T_result.shape)
print("Tmin idx:", [int(i) for i in idx], "Tmin:", float(T_result[idx]))
print("is nozzle there?", bool(nozzle_mask[idx]))
print("is boundary?", int(isn[idx]))
gi = g[:, idx[0], idx[1], idx[2]]
print("g_i min/max:", float(cp.min(gi)), float(cp.max(gi)), "sum:", float(cp.sum(gi)))
print("mean T:", float(cp.mean(T_result)), "min:", float(cp.min(T_result)))

cp.min(T) * C_T-273.13



In [ ]:

rhoc
Pc
Tc
T = state["T"]
rho = state["rho"]
P = state['P']

for i in range(len(rho)):
    if T[i] newP[i] < Pc and 

T_flat = (T * C_T - 273.15).ravel()  # Celsius
P_flat = (P * C_P / 1e6).ravel()     # MPa

plt.scatter(T_flat, P_flat, c=phase_index, cmap='coolwarm', s=1)

In [ ]:
deltaP = newP[25, 25, 200] - newP[25, 25, 219]
deltaT = T[25, 25, 40] - T[25, 25, 60]
JT = (deltaT/deltaP)
C_JT_unit = C_T / C_P * 1e6
print(f'JT: {JT} lb')
print(f'JT: {JT * C_JT_unit} K/MPa')
print(f'current mean T: {cp.mean(T) * C_T} K ')
print(f'current min T: {cp.min(T) * C_T} K ')
print(f'initial T: {T_inlet * C_T} K ')
print(f'meanP: {cp.mean(newP) * C_P/ 1e6} MPa')
print(f'JT in NIST: {2.2} K/MPa')



In [ ]:
rho_last  = cp.mean(rho[:, :, -1])
rho_prev  = cp.mean(rho[:, :, -2])
print("rho last/prev:", float(rho_last), float(rho_prev))

ux_last = cp.mean(u[0, :, :, -1])
ux_prev = cp.mean(u[0, :, :, -2])
print("ux last/prev:", float(ux_last), float(ux_prev))
print(cp.shape(inlet_mask))

In [ ]:
plt.imshow(cp.asnumpy(index[:,ny//2,:]))
plt.colorbar()
plt.show()
maxMa = (cp.sqrt(u[0]**2+u[1]**2+u[2]**2))/cs
maxMa = (cp.sqrt(u[0]**2+u[1]**2+u[2]**2))/cs
plt.imshow(cp.asnumpy(maxMa[:,ny//2,:]))
plt.colorbar()
plt.show()
plt.imshow(cp.asnumpy(newP[:,ny//2,:])*C_P)
plt.colorbar()
plt.show()

In [ ]:
Ma_distribution = (cp.sqrt(u[0]**2+u[1]**2+u[2]**2))/cs
plt.title('Mach at XZ plane')
plt.imshow(cp.asnumpy(Ma_distribution[:,ny//2,:]), cmap='magma', origin='lower')
plt.clim(0,1.0)
plt.colorbar(orientation='horizontal')
plt.show()